## Data Processing

In [1]:
import re, numpy as np, os, sys, pandas
from pymatgen import Composition
import data_utils
from pymatgen import Composition
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from matminer.utils.conversions import str_to_composition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

/home/vgf3011/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.unsupervised module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [2]:
oqmd_data_path = 'sample_train_set.csv'
oqmd_data = pandas.read_csv(oqmd_data_path)
#oqmd_data.info()

In [3]:
oqmd_data.describe()

,H,Li,Be,B,C,N,O,F,Na,Mg,...,Tl,Pb,Bi,Ac,Th,Pa,U,Np,Pu,formation_energy_peratom
count,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,...,451.000000,451.00000,451.000000,451.0,451.000000,451.0,451.000000,451.000000,451.000000,451.000000
mean,0.014502,0.003289,0.001109,0.004009,0.008340,0.020356,0.113170,0.010391,0.002525,0.005897,...,0.006633,0.00630,0.024078,0.0,0.004398,0.0,0.003827,0.000443,0.000739,-0.973250
std,0.073815,0.035862,0.023544,0.030741,0.049105,0.086824,0.213342,0.078004,0.027131,0.041537,...,0.048883,0.04414,0.086991,0.0,0.034864,0.0,0.031177,0.009418,0.015696,0.810400
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-3.268740
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-1.457170
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-0.738420
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-0.359045
max,0.800000,0.500000,0.500000,0.333333,0.428571,0.666667,0.750000,0.800000,0.333333,0.384615,...,0.666667,0.40000,0.666667,0.0,0.333333,0.0,0.333333,0.200000,0.333333,0.648630


In [4]:
%%time
oqmd_data['comp_obj'] = oqmd_data['pretty_comp'].apply(lambda x: Composition(x))

CPU times: user 13.6 ms, sys: 0 ns, total: 13.6 ms
Wall time: 13.3 ms


In [5]:
%%time
oqmd_data['composition'] = oqmd_data['comp_obj'].apply(lambda x: x.reduced_formula)

CPU times: user 34.6 ms, sys: 0 ns, total: 34.6 ms
Wall time: 34.2 ms


In [6]:
oqmd_data['nelems'] = oqmd_data['comp_obj'].apply(lambda x: len(x))

In [7]:
oqmd_data.groupby('nelems').count()

,pretty_comp,H,Li,Be,B,C,N,O,F,Na,...,Bi,Ac,Th,Pa,U,Np,Pu,formation_energy_peratom,comp_obj,composition
nelems,,,,,,,,,,,,,,,,,,,,,
2,176,176,176,176,176,176,176,176,176,176,...,176,176,176,176,176,176,176,176,176,176
3,245,245,245,245,245,245,245,245,245,245,...,245,245,245,245,245,245,245,245,245,245
4,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [8]:
oqmd_data.query('nelems > 1', inplace=True)
print (oqmd_data.groupby('nelems').count())
oqmd_data.shape

        pretty_comp    H   Li   Be    B    C    N    O    F   Na  ...   Bi  \
nelems                                                            ...        
2               176  176  176  176  176  176  176  176  176  176  ...  176   
3               245  245  245  245  245  245  245  245  245  245  ...  245   
4                29   29   29   29   29   29   29   29   29   29  ...   29   
5                 1    1    1    1    1    1    1    1    1    1  ...    1   

         Ac   Th   Pa    U   Np   Pu  formation_energy_peratom  comp_obj  \
nelems                                                                     
2       176  176  176  176  176  176                       176       176   
3       245  245  245  245  245  245                       245       245   
4        29   29   29   29   29   29                        29        29   
5         1    1    1    1    1    1                         1         1   

        composition  
nelems               
2               176  
3       

(451, 91)

In [9]:
%%time
oqmd_data['comp_dict'] = oqmd_data['composition'].apply(lambda x: data_utils.parse_formula(x))

CPU times: user 8.02 ms, sys: 0 ns, total: 8.02 ms
Wall time: 7.81 ms


In [10]:
%%time
elements_present = set()
for i, row in oqmd_data.iterrows():
    elements_present |= set(row['comp_dict'].keys())
print (elements_present)

{'Tl', 'Na', 'Re', 'Cu', 'Pt', 'Al', 'Au', 'Nd', 'Fe', 'Sb', 'Si', 'Pu', 'Sr', 'Ge', 'Tm', 'Lu', 'Ni', 'Kr', 'Sc', 'Zr', 'Ho', 'Sn', 'Ir', 'Co', 'Hg', 'Os', 'Ce', 'Mo', 'Li', 'Np', 'Ti', 'V', 'U', 'Y', 'S', 'Br', 'In', 'Cl', 'Yb', 'Sm', 'O', 'As', 'N', 'Ca', 'Mg', 'Rb', 'F', 'Th', 'Pr', 'P', 'Er', 'Rh', 'Ba', 'Pb', 'H', 'Bi', 'Tb', 'W', 'Be', 'Ag', 'Pd', 'Ta', 'Hf', 'Dy', 'Nb', 'Ga', 'Cd', 'Zn', 'Tc', 'B', 'Te', 'Cr', 'C', 'Se', 'Mn', 'I', 'K'}
CPU times: user 52.4 ms, sys: 669 µs, total: 53.1 ms
Wall time: 52 ms


In [11]:
elements = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
            'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
            'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 
            'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
            'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
            'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
            'Ds', 'Rg', 'Cn']


elements_tl = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

elem_pos = dict()
i=0
for el in elements:
   elem_pos[el] = i
   i+=1

In [12]:
print (elements_present, len(elements_present))
print ([e for e in elements_present if e not in elements_tl])
print ([e for e in elements_tl if e not in elements_present])

{'Tl', 'Na', 'Re', 'Cu', 'Pt', 'Al', 'Au', 'Nd', 'Fe', 'Sb', 'Si', 'Pu', 'Sr', 'Ge', 'Tm', 'Lu', 'Ni', 'Kr', 'Sc', 'Zr', 'Ho', 'Sn', 'Ir', 'Co', 'Hg', 'Os', 'Ce', 'Mo', 'Li', 'Np', 'Ti', 'V', 'U', 'Y', 'S', 'Br', 'In', 'Cl', 'Yb', 'Sm', 'O', 'As', 'N', 'Ca', 'Mg', 'Rb', 'F', 'Th', 'Pr', 'P', 'Er', 'Rh', 'Ba', 'Pb', 'H', 'Bi', 'Tb', 'W', 'Be', 'Ag', 'Pd', 'Ta', 'Hf', 'Dy', 'Nb', 'Ga', 'Cd', 'Zn', 'Tc', 'B', 'Te', 'Cr', 'C', 'Se', 'Mn', 'I', 'K'} 77
[]
['Ru', 'Xe', 'Cs', 'La', 'Pm', 'Eu', 'Gd', 'Ac', 'Pa']


In [13]:
oqmd_data['comp_fractions'] = oqmd_data['comp_dict'].apply(lambda x: data_utils.get_fractions(x))

In [14]:
print (oqmd_data[:1])
#oqmd_data['solution_term'] = oqmd_data['comp_obj'].apply(lambda x: compute_mixing_term(x))
print (oqmd_data.shape)
oqmd_data = oqmd_data[~oqmd_data['comp_fractions'].isnull()]
print (oqmd_data.shape)

  pretty_comp    H   Li   Be    B    C    N    O    F   Na  ...   Pa    U  \
0      ZrGeTe  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

    Np   Pu  formation_energy_peratom      comp_obj  composition  nelems  \
0  0.0  0.0                   -0.8014  (Zr, Ge, Te)       ZrGeTe       3   

                                           comp_dict  \
0  {'Zr': 0.3333333333333333, 'Ge': 0.33333333333...   

                                      comp_fractions  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  

[1 rows x 93 columns]
(451, 93)
(451, 93)


In [15]:
for i,e in enumerate(elements_tl):
    oqmd_data[e] = [ x[i] for x in oqmd_data['comp_fractions']]
oqmd_data.shape

(451, 93)

In [16]:
oqmd_data.columns

Index(['pretty_comp', 'H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg',
       'Al', 'Si', 'P', 'S', 'Cl', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn',
       'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
       'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
       'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm',
       'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
       'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th',
       'Pa', 'U', 'Np', 'Pu', 'formation_energy_peratom', 'comp_obj',
       'composition', 'nelems', 'comp_dict', 'comp_fractions'],
      dtype='object')

In [17]:
feature_calculators = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

In [18]:
feature_labels = feature_calculators.feature_labels()

In [19]:
print (feature_labels, len(feature_labels))

['0-norm', '2-norm', '3-norm', '5-norm', '7-norm', '10-norm', 'MagpieData minimum Number', 'MagpieData maximum Number', 'MagpieData range Number', 'MagpieData mean Number', 'MagpieData avg_dev Number', 'MagpieData mode Number', 'MagpieData minimum MendeleevNumber', 'MagpieData maximum MendeleevNumber', 'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber', 'MagpieData avg_dev MendeleevNumber', 'MagpieData mode MendeleevNumber', 'MagpieData minimum AtomicWeight', 'MagpieData maximum AtomicWeight', 'MagpieData range AtomicWeight', 'MagpieData mean AtomicWeight', 'MagpieData avg_dev AtomicWeight', 'MagpieData mode AtomicWeight', 'MagpieData minimum MeltingT', 'MagpieData maximum MeltingT', 'MagpieData range MeltingT', 'MagpieData mean MeltingT', 'MagpieData avg_dev MeltingT', 'MagpieData mode MeltingT', 'MagpieData minimum Column', 'MagpieData maximum Column', 'MagpieData range Column', 'MagpieData mean Column', 'MagpieData avg_dev Column', 'MagpieData mode Column', 'Magpi

In [20]:
%%time
oqmd_data = feature_calculators.featurize_dataframe(oqmd_data, col_id='comp_obj');


CPU times: user 359 ms, sys: 276 ms, total: 635 ms
Wall time: 678 ms


In [21]:
oqmd_data[feature_labels].shape

(451, 145)

In [22]:
oqmd_data = oqmd_data[~oqmd_data[feature_labels].isnull().any(axis=1)]
print (oqmd_data.shape)

(451, 238)


In [23]:
oqmd_data.describe()

,H,Li,Be,B,C,N,O,F,Na,Mg,...,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,max ionic char,avg ionic char
count,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,...,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000,451.000000
mean,0.014502,0.003289,0.001109,0.004009,0.008340,0.020356,0.113170,0.010391,0.002525,0.005897,...,147.995565,108.522974,57.563808,61.567627,1.906335,2.966925,4.620782,1.372457,0.341089,0.064993
std,0.073815,0.035862,0.023544,0.030741,0.049105,0.086824,0.213342,0.078004,0.027131,0.041537,...,60.629072,43.493470,25.110998,56.334373,0.171937,0.816071,3.313917,2.192619,0.238241,0.046884
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,8.000000,0.000000,2.000000,1.111111,0.000000,0.000000,0.000000,0.000025,0.000006
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,124.000000,81.000000,38.500000,12.000000,1.833333,2.571429,1.666667,0.000000,0.137763,0.023898
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,161.000000,104.250000,59.644970,64.000000,2.000000,3.000000,4.000000,0.000000,0.319141,0.057395
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,211.000000,133.142857,73.162943,70.000000,2.000000,3.519231,7.500000,2.800000,0.532919,0.100665
max,0.800000,0.500000,0.500000,0.333333,0.428571,0.666667,0.750000,0.800000,0.333333,0.384615,...,227.000000,227.000000,112.500000,229.000000,2.000000,5.333333,10.000000,14.000000,0.917619,0.200477


In [24]:
properties = set(oqmd_data.columns.tolist()) - (set(feature_labels) | set(elements_present))
print (properties)

{'Ru', 'comp_obj', 'comp_dict', 'Cs', 'La', 'pretty_comp', 'Pm', 'Gd', 'composition', 'nelems', 'Eu', 'Pa', 'comp_fractions', 'Xe', 'Ac', 'formation_energy_peratom'}


In [25]:
for col in ['comp_fractions', 'comp_dict', 'comp_obj']: oqmd_data.drop(col, axis=1, inplace=True)
oqmd_data.shape

(451, 235)

In [26]:
properties = set(oqmd_data.columns.tolist()) - (set(feature_labels) | set(elements_present) |{'pretty_comp'})
print (properties)

{'Ru', 'Cs', 'La', 'Pm', 'Gd', 'composition', 'nelems', 'Eu', 'Pa', 'Xe', 'Ac', 'formation_energy_peratom'}


In [27]:
oqmd_data = oqmd_data[['pretty_comp']+elements_tl+list(feature_labels)+list(properties)]

In [28]:
print (oqmd_data.columns)
print (oqmd_data.shape)

Index(['pretty_comp', 'H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na',
       ...
       'La', 'Pm', 'Gd', 'composition', 'nelems', 'Eu', 'Pa', 'Xe', 'Ac',
       'formation_energy_peratom'],
      dtype='object', length=244)
(451, 244)


In [30]:
oqmd_data.to_csv('sample_dataset.csv', index=False)